Notebook for processing the geometric calibration of each channel of OROS, and cross-channel stereo calibration.

# Process

First we will process, for each channel, the intrinsic and extrinsic camera matrices from images of the square 10x10 calibration target oriented at mutiple positions.
We will then draw the epipolar lines on these images, showing, for each channel, the epipolar line of each other channel in turn.

Then, we will experiment with drawing the epipolar lines over the sample-region mounted calibration target, that has been imaged at multiple positions. Again, we will draw epipolar lines for each positions, for each channel with each other channel.

Steps:
1. For each position, load images for each channel.
2. Perform averaging and average dark subtraction
3. For each position, find the calibration target points
4. For all positions, and for each channel, solve for camera matrices
5. For each position, and for each channel, find and draw epipolar lines for each other channel

# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import orochi_sim_proc as osp
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# get the location of this notebook
import os
from pathlib import Path
notebook_dir = Path(os.path.abspath(''))
output_dir = Path(notebook_dir, 'stereo_calibration_processing_22082023_outputs')
output_dir.mkdir(parents=True, exist_ok=True)

# Loading images

In [ ]:
# get the list of positions
# get the list of the cameras
# for each position, for each camera, load the image
# find the calibration points, store in a dictionary indexed by camera
# for each camera, calibrate to get intrinsic and extrinsic matrices


In [ ]:
n_pos = 35
positions = {}
for pos in range(n_pos):
    print(f'Position {pos+1} of {n_pos}')
    pos_dir = f'geom_calibration_pos_{pos}_22082023'
    dark_dir = f'geom_calibration_dark_22082023'
    geocs = osp.load_geometric_calibration(subject=pos_dir, dark=dark_dir, caption=f'Target Position {pos+1} of {n_pos}')
    positions[pos] = geocs
# del positions[7]
# del positions[0]
# del positions[1]
# del positions[9]
# note this is a very long process when we have to do it for all 35 positions

# Getting Checkerboard Corner Points

In [ ]:
positions[22]

In [ ]:
corners = {}
chkrbrd = (5,7,0.015)
for pos in positions:
    print(f'Position {pos+1} of {n_pos}')
    geocs = positions[pos]
    corner_set = osp.checkerboard_calibration(geocs, chkrbrd=chkrbrd, caption=f'Target Position {pos+1}')
    corners[pos] = corner_set

# Solve for Intrinsic Calibration Matrix

In [ ]:
import cv2

In [ ]:
position_lst = list(corners.keys())
channels = list(corners[list(position_lst)[0]].keys())
for channel in channels:
    imgpoints = []
    objpoints = []
    valid_positions = []
    for position in position_lst:
        # get the corners for the current channel and position
        # TODO skip if none
        pos_points = corners[position][channel].corner_points
        if pos_points is not None:
            valid_positions.append(position)
            imgpoints.append(pos_points)
            objpoints.append(corners[position][channel].object_points)
            shape = corners[position][channel].img_ave.shape[::-1]
    # calibrate the channel
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)        
    for n,position in enumerate(valid_positions): 
        corners[position][channel].calibration_error = ret
        corners[position][channel].mtx = mtx
        corners[position][channel].dist = dist
        corners[position][channel].rvec = rvecs[n]
        corners[position][channel].tvec = tvecs[n]    
        corners[position][channel].camera_intrinsic_properties()    
        r_mtx, _ = cv2.Rodrigues(rvecs[n])
        e_mtx = np.c_[r_mtx, tvecs[n]]
        corners[position][channel].p_mtx = np.matmul(mtx, e_mtx)     

# Check the Distortion Coefficients

In [ ]:
position_lst = list(corners.keys())
channels = list(corners[list(position_lst)[0]].keys())
for position in position_lst:
    fig, ax = osp.grid_plot(f'Distortion Difference Check: Position {position}')
    for channel in channels:
        test_cam = corners[position][channel]
        test_cam.check_distortion(ax=ax[test_cam.camera])
    osp.show_grid(fig, ax)

# Check the calibration by drawing the axes on the geocs images - and export

In [ ]:
position_lst = list(corners.keys())
for position in position_lst:
    fig, ax = osp.grid_plot(f'Calibration Check: Position {pos+1} of {len(position_lst)}')
    for channel in channels:        
        cam_pos = corners[position][channel]
        cam_pos.export_calibration(dir = output_dir)
        cam_pos.project_axes(ax=ax[cam_pos.camera], corner_roi=False)
    osp.show_grid(fig, ax)

# Stereo Calibration

To do, ignore pairing if points are missing

In [ ]:
cam_pairs = {}
channels = list(corners[list(position_lst)[0]].keys())
position_lst = list(corners.keys())
for channel in channels:
    cam_pairs[channel] = {}
    for sub_channel in channels:
        cam_pairs[channel][sub_channel] = {}
        for position in position_lst:
            if corners[position][channel].corner_points is not None:
                if corners[position][sub_channel].corner_points is not None:
                    cam_pairs[channel][sub_channel][position] = None

for channel in channels:
    for sub_channel in channels:
        valid_positions = list(cam_pairs[channel][sub_channel].keys())
        print(valid_positions)
        a_imgpoints = []
        a_objpoints = []    
        b_imgpoints = []
        b_objpoints = []
        for position in valid_positions:
            a_cam = corners[position][channel]       
            a_imgpoints.append(a_cam.corner_points)
            a_objpoints.append(a_cam.object_points)    
            b_cam = corners[position][sub_channel]
            b_imgpoints.append(b_cam.corner_points)
            b_objpoints.append(b_cam.object_points)
        for position in valid_positions:
            a_cam = corners[position][channel]    
            b_cam = corners[position][sub_channel]   
            cam_pair = osp.StereoPair(a_cam, b_cam)
            # set the points from all target positions
            cam_pair.src_pts = a_imgpoints
            cam_pair.dst_pts = b_imgpoints
            # check a_obj_points == b_objpoints
            cam_pair.obj_pts = a_objpoints
            # perform the stereo calibration
            print(f'Pos. {position} A: {a_cam.camera}, B: {b_cam.camera}')            
            pair_error = cam_pair.calibrate_stereo()
            cam_pair.export_calibration(dir=output_dir)
            cam_pairs[channel][sub_channel][position] = cam_pair

# Epipolar Geometry

For each position, and for each channel in turn (starting with the #7 550 nm position), draw the epipolar lines for each other channel.

In [ ]:
for position in position_lst:
    for channel in channels:        
        home_cam = corners[position][channel]        
        fig_fs_lines, ax_fs_lines = osp.grid_plot(f'Target Position {position+1} for Channel {home_cam.camera}: {int(home_cam.cwl)} nm Epipolar Lines from Fundamental Matrix')
        for sub_channel in channels:                        
            if position in cam_pairs[channel][sub_channel]:
                cam_pair = cam_pairs[channel][sub_channel][position]
                src_lines, dst_lines = cam_pair.compute_epilines('F')
                cam_pair.draw_epilines(view='dst', ax=ax_fs_lines[cam_pair.dst.camera])
                cam_pair.draw_epilines(view='src', ax=ax_fs_lines[8])            
        osp.show_grid(fig_fs_lines, ax_fs_lines)

# Stereo Rectify

In [ ]:
position_lst = list(corners.keys())
for position in position_lst: # use the first position only for now
    for channel in [channels[0]]: # just use first channel for development        
        home_cam = corners[position][channel]        
        fig_fs_lines, ax_fs_lines = osp.grid_plot(f'Target Position {position+1} for Channel {home_cam.camera}: {int(home_cam.cwl)} nm Stereo Rectification')
        for sub_channel in channels:    
            valid_positions = list(cam_pairs[channel][sub_channel].keys())
            if position in valid_positions:        
                cam_pair = cam_pairs[channel][sub_channel][position]          
                print(f'Rectifying {channel} and {sub_channel} i.e. {cam_pair.src.camera} and {cam_pair.dst.camera}') 
                # print(cam_pair.r_mtx)
                # print(np.sqrt(np.sum(np.array(cam_pair.t_mtx)**2)))            
                cam_pair.rectify(ax=ax_fs_lines[cam_pair.dst.camera])            
        osp.show_grid(fig_fs_lines, ax_fs_lines)

# Stereo Matching for Disparity Map
 Compute a disparity map from each image pair.

In [ ]:
for position in position_lst:
    for channel in [channels[0]]:
        fig_disp, ax_disp = osp.grid_plot(f'Disparity for Position {position+1} for Channel {channel}')
        for sub_channel in channels:                
            cam_pair = cam_pairs[position][channel][sub_channel]
            points3D = cam_pair.compute_disparity(ax=ax_disp[cam_pair.dst.camera])            
        osp.show_grid(fig_disp, ax_disp)

# Projecting from Image to 3D

Now we will use the depth maps and the Q matrix, and the new projection matrices, to attempt to project from the images to 3D space.

In [ ]:
# lets try to triangulate with projection matrices, computed from the intrinsic x extrinsic matrices for each camera, found through calibration
# position = 2
for position in position_lst:
    for channel in [channels[0]]:
# channel = '0_850'
        fig_rec, ax_rec = osp.grid_plot(f'Target 3D Reconstruction for Position {position+1} for Channel {channel}', projection='3d')
        for sub_channel in channels:                
            cam_pair = cam_pairs[position][channel][sub_channel]
            points3D = cam_pair.depth_from_disparity()
            cam_pair.plot_3D_points(ax=ax_rec[cam_pair.dst.camera])
        osp.show_grid(fig_rec, ax_rec)

# Triangulation & Depth Maps

In [ ]:
# lets try to triangulate with projection matrices, computed from the intrinsic x extrinsic matrices for each camera, found through calibration
# position = 2
for position in position_lst:
    for channel in channels:
# channel = '0_850'
        fig_rec, ax_rec = osp.grid_plot(f'Target 3D Reconstruction for Position {position+1} for Channel {channel}', projection='3d')
        for sub_channel in channels:                
            cam_pair = cam_pairs[position][channel][sub_channel]
            points3D = cam_pair.compute3D()
            cam_pair.plot_3D_points(ax=ax_rec[cam_pair.dst.camera])
        osp.show_grid(fig_rec, ax_rec)
    

In [ ]:
position = 2
# for position in position_lst:
    # for channel in channels:
channel = '0_850'
fig_fs_lines, ax_fs_lines = osp.grid_plot(f'Target Position {position+1} for Channel {home_cam.camera}: {int(home_cam.cwl)} nm Epipolar Lines from Fundamental Matrix')
fig_rec, ax_rec = osp.grid_plot(f'Target Position {position+1} for Channel {channel} Rectified images')
for sub_channel in channels:                
    cam_pair = cam_pairs[position][channel][sub_channel]
    cam_pair.draw_epilines(view='dst', ax=ax_fs_lines[cam_pair.dst.camera])
    cam_pair.draw_epilines(view='src', ax=ax_fs_lines[8])
    cam_pair.rectify(ax=ax_rec[cam_pair.dst.camera])
osp.show_grid(fig_fs_lines, ax_fs_lines)
osp.show_grid(fig_rec, ax_rec)